In [9]:
import pickle
import numpy as np

results = pickle.load(open('/home/test4/code/EventBenchmark/lib/pytracking/pytracking/tracking_results_rt_raw/atom/default_000/airplane5.pkl', 'rb'))


In [3]:
results_raw = results.get('results_raw', None)


In [12]:
results.keys()

dict_keys(['results_raw', 'out_timestamps', 'in_timestamps', 'runtime'])

In [26]:
results_raw = results.get('results_raw', None)
# timestamps = results['timestamps']
gt_anno_t_path = '/home/test4/code/EventBenchmark/data/EventSOT/EventSOT500/EventSOT500/anno_t/airplane5.txt'
gt_anno_t = np.loadtxt(gt_anno_t_path, delimiter=' ')
# timestamps[0]=0
# 读入GT 每行的格式是[timestamp, x1, y1, x2, y2]
# 对于GT的每一行，找到对应的Prediction，逻辑是取Prediction中out_timestamp<=gt timestamp中的最后一个
def find_last_pred(gt_t, pred_raw):
    pred_timestamps = pred_raw['out_timestamps']
    pred_timestamps[0] = 0
    gt_t = gt_t*1e6
    # print(gt_t, pred_timestamps[-1])
    # assert abs(gt_t - pred_timestamps[-1]) < 100  # time unit:s
    last_pred_idx = np.searchsorted(pred_timestamps, gt_t)-1
    pred_results = pred_raw['results_raw']
    pred_last_result = pred_results[last_pred_idx]
    pred_last_time = pred_timestamps[last_pred_idx]
    # print(gt_t, pred_last_time)
    return pred_last_result
pred_final = []
for line in gt_anno_t:
    gt_t = line[0]
    gt_label = line[1:]
    pred_label = find_last_pred(gt_t, results)
    pred_bbox = pred_label
    pred_final.append(pred_bbox)

In [27]:
np.savetxt('output.txt',pred_final,fmt='%d',delimiter='\t')

### version3.0

In [5]:
import os
import sys
import importlib
env_path = os.path.join('/home/test4/code/EventBenchmark/lib/pytracking/')
if env_path not in sys.path:
    sys.path.append(env_path)

experiment_module = 'myexperiments'
experiment_name = 'esot2_streaming_1'
expr_module = importlib.import_module('pytracking.experiments.{}'.format(experiment_module))
expr_func = getattr(expr_module,experiment_name)
trackers, dataset, stream_setting = expr_func()